In [1]:
#!pip install praw pandas nltk spacy pyspellchecker

  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 3.7 MB/s eta 0:00:005.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 5.8 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 4.7 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 4.6 MB/s eta 0:00:007.0 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 2.8 MB/s eta 0:00:000:00:01
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 3.7 MB/s eta 0:00:006.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 4.7 MB/s eta 0:00:00m eta 0:00:010:01
Using cached click-8.1.8-py3-none-

In [3]:
import praw

reddit = praw.Reddit(
    client_id="U1cbjxQXMCcRU1Bvet60RA",
    client_secret="YKDhzchaWbAvglen1qPBCITEykvHNQ",
    user_agent="PLN"
)

In [4]:
import json
from tqdm import tqdm

SUBREDDITS = ["nba", "deeplearning", "nasa", "movies", "politics", "science"]
MAX_THREADS_PER_SUBREDDIT = 20
COMMENTS_PER_THREAD = 50

def scrape_reddit():
    for subreddit_name in SUBREDDITS:
        subreddit = reddit.subreddit(subreddit_name)
        threads_data = []
        
        # Buscar hilos populares (ajustar parámetros según necesidad)
        for thread in tqdm(subreddit.top(limit=MAX_THREADS_PER_SUBREDDIT), desc=f"Extrayendo de r/{subreddit_name}"):
            if thread.over_18:  # Saltar contenido NSFW
                continue
                
            comments = []
            thread.comments.replace_more(limit=0)  # Limitar comentarios anidados
            for comment in thread.comments[:COMMENTS_PER_THREAD]:
                comments.append({
                    "user": str(comment.author),
                    "comment": comment.body,
                    "score": comment.score,
                    "date": comment.created_utc
                })
            
            threads_data.append({
                "flair": thread.link_flair_text,
                "title": thread.title,
                "author": str(thread.author),
                "date": thread.created_utc,
                "score": thread.score,
                "description": thread.selftext,
                "comments": comments
            })
        
        # Guardar en JSON
        with open(f"subreddit_{subreddit_name}.json", "w", encoding="utf-8") as f:
            json.dump(threads_data, f, indent=4, ensure_ascii=False)

scrape_reddit()

Extrayendo de r/nba: 20it [01:18,  3.93s/it]
Extrayendo de r/deeplearning: 20it [00:10,  1.96it/s]
Extrayendo de r/nasa: 20it [00:36,  1.85s/it]
Extrayendo de r/movies: 20it [01:23,  4.17s/it]
Extrayendo de r/politics: 20it [01:23,  4.20s/it]
Extrayendo de r/science: 20it [01:15,  3.76s/it]


In [9]:
import re
import nltk
from nltk.corpus import stopwords
from spellchecker import SpellChecker

# Cargar datos de un subreddit (ejemplo: "technology")
with open("subreddit_nba.json", "r", encoding="utf-8") as f:
    threads_data = json.load(f)

nltk.download('stopwords') 

spell = SpellChecker(language='en')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Eliminar URLs, menciones y caracteres especiales
    text = re.sub(r"http\S+|@\w+|[^a-zA-ZáéíóúñÁÉÍÓÚÑ ]", "", text)
    
    # Corrección ortográfica
    words = text.split()
    corrected = [spell.correction(word) if spell.correction(word) is not None else word for word in words]
    
    # Eliminar stopwords y lematización (usar spaCy para mejor precisión)
    cleaned = [word for word in corrected if word.lower() not in stop_words]
    
    return " ".join(cleaned)

# Aplicar a todos los comentarios y descripciones
for thread in threads_data:
    thread["description"] = clean_text(thread["description"])
    for comment in thread["comments"]:
        comment["comment"] = clean_text(comment["comment"])

[nltk_data] Downloading package stopwords to /home/pablo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


KeyboardInterrupt: 